In [4]:
!git clone https://github.com/ju-leon/ClassifyAmpules.git
%cd ClassifyAmpules

fatal: destination path 'ClassifyAmpules' already exists and is not an empty directory.
/content/ClassifyAmpules


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Updating

If you make changes to the repository, use this command to update your files

In [ ]:
!git pull

# Setup

Specify the paths you want to use

In [6]:
source_path = "/content/drive/My Drive/Hackaton/PIM_Dataset"
cache_path = "/content/drive/My Drive/Hackaton/PIM_combi"
out_path =  "/content/drive/My Drive/Hackaton/combined"

source_path_good = "'" + source_path + "/good_piece/" + "'"
source_path_bad = "'" + source_path + "/bad_piece" + "'"

cache_path_good = "'" + cache_path + "/good_piece/" + "'"
cache_path_bad = "'" + cache_path + "/bad_piece/" + "'"

out_path_good = "'" + out_path + "/good_piece/" + "'"
out_path_bad = "'" + out_path + "/bad_piece/" + "'"
out_path = "'" + out_path + "'"

# Preprocessing

First, we need to scale down our data. 

In [ ]:
!python3 preprocess.py $source_path_good $cache_path_good
!python3 preprocess.py $source_path_bad $cache_path_bad

# Combining

Now we combine 3 images into one.

In [8]:
!python3 combine.py $cache_path_good $out_path_good
!python3 combine.py $cache_path_bad $out_path_bad

100% 1680/1680 [00:00<00:00, 990501.93it/s]
100% 560/560 [01:50<00:00,  5.06it/s]


# Filter by bad piece

In [13]:
from shutil import copyfile
import os
path = "/content/drive/My Drive/Hackaton/combined/good_piece/"

valid_images = ".png"
for f in os.listdir(path):
    if f.endswith(valid_images):
        copyfile(path + f, "/content/drive/My Drive/Hackaton/all_classes/good/" + f)


# Training

In [ ]:
!python3 train.py $out_path

Using TensorFlow backend.
2020-07-02 21:36:30.177344: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Found 2677 images belonging to 2 classes.
Found 1147 images belonging to 2 classes.
(100, 100, 3)
/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
2020-07-02 21:36:32.532718: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-02 21:36:32.572278: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-02 21:36:32.572942: I tensorflow/core/common_runtime/gpu/gpu_